In [7]:
import string
from gensim import models
from gensim.corpora import Dictionary, MmCorpus
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.collocations import *
import pandas as pd
from langdetect import detect

import pattern.fr

import pattern.en
import re
import string

import pyLDAvis.gensim as gensimvis
import pyLDAvis
import unicodedata

In [10]:
sw = pd.read_excel('liste noire.xlsx')
EMAIL_REGEX = re.compile(r"[a-z0-9\.\+_-]+@[a-z0-9\._-]+\.[a-z]*")
FILTER_REGEX = re.compile(r"[^a-z 'éèëêà#]")
TOKEN_MAPPINGS = [(EMAIL_REGEX, "#email"), (FILTER_REGEX, ' ')]



def remove_accents(s):
   # s = unicode(s,"utf8","replace")
    s=unicodedata.normalize('NFD',s)
    return s.encode('ASCII','ignore')


def nltk_stopwords(lang):
    if lang == 'fr':
        return set(nltk.corpus.stopwords.words('french'))
    if lang == 'en':
        return set(nltk.corpus.stopwords.words('english'))

def tokenize_line(line,lang):
    
    res = line.lower()
    #res = remove_accents(res)
    #for regexp, replacement in TOKEN_MAPPINGS:
    #   res = regexp.sub(replacement, res)
    if lang == "fr":
        tok = pattern.fr.parse(res, tokenize = True, lemmata=True).split()
        
    if lang == "en":
        tok = pattern.en.parse(res, tokenize = True, lemmata=True).split()
    
    tokens = [item[4] for sublist in tok for item in sublist if item[1] not in ['.',',',':',';','-',"'",'"','IN','CC','PRP', 'DT','RB',')','('] and item[4] not in sw]
    
    return tokens 

    # Gestion des collocations :
    #bigram_measures = nltk.collocations.BigramAssocMeasures()
    #trigram_measures = nltk.collocations.TrigramAssocMeasures()
    #finder = BigramCollocationFinder.from_words(tokens).nbest(bigram_measures.pmi, len(tokens))
    #return [x[0]+' '+x[1] for x in finder]+tokens
    

def prep_corpus(docs, lang, stopwords, no_below=5, no_above=0.5):
    print('Building dictionary...')
    dictionary = Dictionary(docs)
    stopword_ids = map(dictionary.token2id.get, stopwords)
    dictionary.filter_tokens(stopword_ids)
    dictionary.filter_extremes(no_below=no_below, no_above=no_above, keep_n=10000)
    dictionary.compactify()

    print('Building corpus...')
    corpus = [dictionary.doc2bow(doc) for doc in docs]

    return dictionary, corpus


In [11]:
tokenize_line('test deux trois',"fr")

NameError: global name 'sw' is not defined

In [12]:
s1=pd.read_csv('bnp.csv', encoding = 'utf-8').drop_duplicates()
s2=pd.read_csv('data1.csv', encoding = 'utf-8').drop_duplicates()
s3=pd.read_csv('data2.csv', encoding = 'utf-8').drop_duplicates()
s4=pd.read_csv('data3.csv', encoding = 'utf-8').drop_duplicates()
reviews = pd.concat([s1, s2, s3])
reviews['language'] = reviews['reviewDescription'].apply(lambda x: detect(x))
reviews = reviews.drop_duplicates('reviewDescription')

In [62]:
reviews_fr = reviews[reviews['language'] == 'fr']
reviews_en = reviews[reviews['language'] == 'en']
#création liste stop words
sw = []
companies = tokenize_line(reviews['company'].drop_duplicates().to_string(),"en")
sw = stopwords.words("english")+stopwords.words("french")+[u'être',u'avoir',u'avoir être',u'etre']+companies

In [105]:
#tokenization:
#reviews_fr['tokens'] = reviews_fr['reviewDescription'].apply(lambda x: tokenize_line(x,'fr'))
reviews_en['tokens'] = reviews_en['reviewDescription'].apply(lambda x: tokenize_line(x,'en'))

C:\Users\merli\Anaconda3\envs\py27\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [106]:
#reviews_fr.to_csv('reviews_fr.csv',encoding = 'utf-8')
reviews_en.to_csv('reviews_en.csv',encoding = 'utf-8')
reviews_en.tokens

###  "Travail en équipe"
[(1, u'0.108*bon + 0.088*dynamique + 0.084*\xe9quipe + 0.069*permettre + 0.063*mum + 0.053*apprentissage + 0.049*connaissance + 0.047*plein + 0.041*\xe9quipe dynamique + 0.017*entente')

### "Créer, Faire, Produire"
(12, u'0.064*faire + 0.043*pouvoir + 0.042*produit + 0.039*eter + 0.034*agr\xe9able + 0.025*excellent + 0.024*apprendre + 0.023*stage + 0.022*formation + 0.022*mum')

### "Ambiance de travail, Culture"
(10, u'0.087*bon + 0.072*entreprise + 0.045*ambiance + 0.039*travail + 0.024*bon ambiance + 0.022*sein + 0.020*exp\xe9rience + 0.020*esprit + 0.016*ambiance travail + 0.013*culture')

### "Métiers - vente produits - tournées courrier"
(11, u'0.127*employ\xe9 + 0.120*vente + 0.072*emploi + 0.066*social + 0.059*comp\xe9tent + 0.055*intense + 0.055*vendre + 0.045*produit + 0.045*changer + 0.034*7') 
(4, u'0.029*travail + 0.024*travailler + 0.024*courrier + 0.024*aspect + 0.017*faire + 0.017*difficile + 0.016*agr\xe9able + 0.016*tourn\xe9e + 0.012*pouvoir + 0.012*tri')

### "Développement / Expérience / Autonomie"
(2, u'0.088*cadre + 0.057*exp\xe9rience + 0.051*d\xe9veloppement + 0.051*bon + 0.050*monde + 0.049*tre + 0.045*autonomie + 0.045*coll\xe8gue + 0.044*savoir + 0.044*nouvel')

### "Evolution, Opportunités"
(3, u'0.033*\xe9volution + 0.027*travail + 0.025*possible + 0.020*\xe9volution possible + 0.016*clients + 0.015*agr\xe9able + 0.014*gestion + 0.014*opportunit\xe9 + 0.012*formation + 0.012*collaborateur')

### "Contrats et Salaires"
(7, u'0.085*travail + 0.055*contrat + 0.029*sympathique + 0.029*demander + 0.029*salaire + 0.029*s\xe9curit\xe9 + 0.028*bon + 0.028*opportunit\xe9 + 0.028*agent + 0.027*bel')

### "Quotidien"
(5, u'0.084*\xe9quipe + 0.074*outil + 0.068*remonter + 0.049*journ\xe9e + 0.048*commencer + 0.047*... + 0.046*travail + 0.042*collaborateur + 0.042*comp\xe9tence + 0.041*journ\xe9e travail')

(9, u'0.056*aimer + 0.051*importer + 0.045*faire + 0.038*travail agr\xe9able + 0.038*agr\xe9able + 0.030*changement + 0.030*\xe9quipe + 0.030*part + 0.029*occuper + 0.029*postes')]

### "Attentes et objectifs Pro"
(14, u'0.159*travail + 0.116*id\xe9e + 0.099*sympa + 0.089*personnel + 0.050*job + 0.049*raison + 0.048*aujourd + 0.048*exiger + 0.048*contribuer + 0.048*atteindre objectif')


In [108]:
dictionary, corpus = prep_corpus((reviews_fr['tokens']),'fr', sw)
#dictionary, corpus = prep_corpus((reviews_en),'en')

lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=15, passes=20)         
vis_data = gensimvis.prepare(lda, corpus, dictionary)
pyLDAvis.display(vis_data)

Building dictionary...
Building corpus...


In [88]:
reviews['reviewDate']

0           6 juillet 2015
1         25 novembre 2015
2         24 novembre 2015
3         21 novembre 2015
4          3 novembre 2015
5          24 octobre 2015
6          16 octobre 2015
7           7 octobre 2015
8           6 octobre 2015
9           5 octobre 2015
10       29 septembre 2015
11       29 septembre 2015
12       26 septembre 2015
13       23 septembre 2015
14       21 septembre 2015
15       18 septembre 2015
16       15 septembre 2015
17       14 septembre 2015
18        9 septembre 2015
19        3 septembre 2015
20            27 août 2015
22            19 août 2015
23             7 août 2015
24             6 août 2015
25             4 août 2015
26             4 août 2015
27         30 juillet 2015
28         27 juillet 2015
29         25 juillet 2015
30         21 juillet 2015
               ...        
15112       9 février 2015
15113       9 février 2015
15114       7 février 2015
15115      22 janvier 2015
15116       7 janvier 2015
15117     18 décembre 2014
1

In [109]:
lsi = models.lsimodel.LsiModel(corpus=corpus, id2word=dictionary, num_topics=20)         
lsi.print_topics(20)

[(0,
  u'0.523*"travail" + 0.308*"bon" + 0.247*"agr\xe9able" + 0.212*"aspect" + 0.201*"faire" + 0.188*"\xe9quipe" + 0.176*"entreprise" + 0.151*"difficile" + 0.143*"journ\xe9e" + 0.133*"clients"'),
 (1,
  u'0.732*"bon" + -0.235*"courrier" + -0.206*"tourn\xe9e" + -0.191*"aspect" + -0.158*"faire" + 0.139*"ambiance" + -0.130*"clients" + -0.128*"journ\xe9e" + -0.126*"difficile" + 0.120*"entreprise"'),
 (2,
  u'0.692*"travail" + -0.323*"entreprise" + -0.276*"bon" + -0.196*"faire" + -0.156*"courrier" + 0.155*"agr\xe9able" + -0.141*"pouvoir" + -0.119*"permettre" + -0.106*"formation" + -0.102*"tourn\xe9e"'),
 (3,
  u'0.443*"courrier" + 0.433*"tourn\xe9e" + 0.303*"bon" + -0.207*"entreprise" + 0.203*"tri" + 0.193*"colis" + -0.155*"formation" + -0.136*"pouvoir" + -0.135*"gestion" + -0.130*"permettre"'),
 (4,
  u'0.461*"aspect" + -0.377*"travail" + -0.375*"entreprise" + 0.318*"agr\xe9able" + 0.261*"bon" + 0.234*"difficile" + 0.214*"aspect agr\xe9able" + 0.200*"aspect difficile" + -0.120*"faire" + -